In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import torch
from transformers import GPT2Tokenizer, GPT2Model
from scipy.stats import uniform

# Mount Google Drive
drive.mount('/content/drive')

# Load the GPT-2 model and tokenizer from Hugging Face
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

# Function to convert text to GPT-2 embeddings
def convert_to_gpt2_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to get GPT-2 embeddings for a text
    def get_gpt2_embeddings(text):
        # Add padding token to tokenizer if it doesn't exist
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).detach().numpy().flatten()

    # Generate embeddings for the text column
    embeddings = df[text_column].apply(lambda x: get_gpt2_embeddings(str(x)))

    # Stack embeddings into a 2D array
    embeddings = np.stack(embeddings.values)

    # Print the dimensions of the embeddings
    print(f"GPT-2 embeddings dimensions for '{text_column}': {embeddings.shape}")

    # Return the embeddings and the original DataFrame
    return embeddings, df

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Define individual model functions
def knn_model(X_train, y_train, X_test, y_test):
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    model = MLPClassifier(max_iter=300)
    model.fit(X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    model = XGBClassifier(eval_metric='mlogloss')
    model.fit(X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    model = AdaBoostClassifier(algorithm='SAMME')
    model.fit(X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    model = CatBoostClassifier(verbose=0)
    model.fit(X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=300)
    model.fit(X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    model = SVC()
    model.fit(X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Define RandomizedSearchCV for hyperparameter tuning for RandomForest
def random_forest_tune(X_train, y_train):
    param_dist = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': uniform(0.01, 0.1),
        'min_samples_leaf': uniform(0.01, 0.1)
    }
    rf = RandomForestClassifier()
    random_search = RandomizedSearchCV(rf, param_dist, n_iter=30, cv=5, verbose=1, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best RandomForest Parameters: {random_search.best_params_}")
    return random_search.best_estimator_

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Specify the text column for GPT-2 embeddings
text_column = 'cleaned_answer'

# Convert text column to GPT-2 embeddings and get the original DataFrame
embeddings, original_data = convert_to_gpt2_embeddings(input_file, text_column)

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.3, random_state=42)

# Run hyperparameter tuning for Random Forest with RandomizedSearchCV
best_rf_model = random_forest_tune(X_train, y_train)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

GPT-2 embeddings dimensions for 'cleaned_answer': (681, 768)
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best RandomForest Parameters: {'max_depth': 30, 'min_samples_leaf': 0.046322832771529945, 'min_samples_split': 0.020914852687255295, 'n_estimators': 100}
Running KNN model...
KNN Model Metrics:
Train Accuracy: 0.6849
Train Precision: 0.6602
Train Recall: 0.6849
Train F1 Score: 0.6639
Train Confusion Matrix:
[[  7   9  16]
 [  4  51  77]
 [  3  41 268]]

Test Accuracy: 0.6634
Test Precision: 0.6046
Test Recall: 0.6634
Test F1 Score: 0.6313
Test Confusion Matrix:
[[  0   3  10]
 [  1   8  32]
 [  2  21 128]]

Running Random Forest model...
Random Forest Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.7415
Test Precision: 0.6534
Test Recall: 0.7415
Test F1 Score: 0.6592
Test Confusion Matrix:
[[  0   1  12]
 [  0   4  37]
 [

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.6293
Test Precision: 0.6460
Test Recall: 0.6293
Test F1 Score: 0.6360
Test Confusion Matrix:
[[  2   4   7]
 [  3  14  24]
 [  4  34 113]]

Running XGBoost model...
XGBoost Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.6829
Test Precision: 0.6437
Test Recall: 0.6829
Test F1 Score: 0.6591
Test Confusion Matrix:
[[  2   2   9]
 [  2   9  30]
 [  3  19 129]]

Running AdaBoost model...
AdaBoost Model Metrics:
Train Accuracy: 0.7500
Train Precision: 0.7431
Train Recall: 0.7500
Train F1 Score: 0.7401
Train Confusion Matrix:
[[ 18  11   3]
 [  1  62  69]
 [  0  35 277]]

Test Accuracy: 0.6244
Test Precision: 0.6093
Test Recall: 0.6244
Test F1 Score: 0.6

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Model Metrics:
Train Accuracy: 0.8887
Train Precision: 0.8883
Train Recall: 0.8887
Train F1 Score: 0.8854
Train Confusion Matrix:
[[ 24   2   6]
 [  3  98  31]
 [  0  11 301]]

Test Accuracy: 0.6293
Test Precision: 0.6411
Test Recall: 0.6293
Test F1 Score: 0.6347
Test Confusion Matrix:
[[  4   2   7]
 [  4  11  26]
 [  3  34 114]]

Running SVM model...
SVM Model Metrics:
Train Accuracy: 0.6555
Train Precision: 0.4296
Train Recall: 0.6555
Train F1 Score: 0.5190
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 132]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.5426
Test Recall: 0.7366
Test F1 Score: 0.6249
Test Confusion Matrix:
[[  0   0  13]
 [  0   0  41]
 [  0   0 151]]

